# Desafio Properati - Limpieza de datos - Grupo 3

En este protecto el desafio es limpiar la base de datos de inmuebles provista por Properati.

El objetivo de la limpieza es dejar listo el dataset para despues poder usarlo para hacer regresiones y calcular el valor de nuevas observaciones.

## ¿Como lo vamos a hacer?
* 1ro Analisis explorativo: Hacer mil pruebas y analizar que estrategias tomar con estos datos.
* 2do Normalizar, corregir y rellenar la informacion que lo permita sin afectar prediciones futuras
* 3ro Quitar todo lo que no nos sirve
* 4to Calcular las variables dummies y mostrar los resultados

## 1. Analisis exploratorio

A partir del analisis exploratorio de los datos, ponemos a prueba algunas de las hipotesis que tendremos en cuenta para estandarizar la información que nos permita predicciones futuras. 
En los casos en los que nuestras hipótesis se corroboran, llevamos a cabo su ejecución en el bloque siguiente. 

In [1]:
# Importo librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re

# usado para pruebas hechas sobre  las urls de imagenes y link a las publicaciones
import requests 
import hashlib 

from IPython.core.display import HTML
%matplotlib inline

In [2]:
# importo archivo
df = pd.read_csv("properatti.csv")
df.shape

(121220, 26)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

In [4]:
df.describe()

,Unnamed: 0,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,121220.000000,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,60609.500000,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,34993.344153,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,0.000000,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,30304.750000,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,60609.500000,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,90914.250000,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,121219.000000,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


#### Trabajo de  Hipotesis sobre m2 con valores invertidos

In [ ]:
# Buscamos los casos en los que la superficie total es menor que la superficie cubierta.
print("Muestra de M2 con superficie total menor que superficie cubierta")
display(df.loc[(df.surface_total_in_m2 < df.surface_covered_in_m2),["surface_total_in_m2","surface_covered_in_m2"]].sample(5))


#Suponemos que los valores de las columnas de superficie total y superficie cubierta pueden estar invertidos por error. 
#Estimamos la división de uno sobre otro para calcular la media de esta diferencia que nos permita corroborar nuestra hipótesis.
df['cubierta_sobre_total'] = df['surface_total_in_m2']/ df['surface_covered_in_m2'] 
df['total_sobre_cubierta'] = df['surface_covered_in_m2']/ df['surface_total_in_m2']
df['valores_invertidos'] = df['surface_covered_in_m2'] < df['surface_total_in_m2']

#Dropeamos los valores iguales para que no afecten el promedio.
#Observamos que son valores similares y que por lo tanto nos permiten asumir que los valores de ambas columnas fueron invertidos.
print("Resumen de la media de la relacion de variables sobre M2 (valores invertidos y no invertidos)")
display(
    df.drop(df.loc[df['surface_total_in_m2'] == df['surface_covered_in_m2']].index)\
    [["valores_invertidos","total_sobre_cubierta","cubierta_sobre_total"]].groupby(['valores_invertidos']).mean()
)

# Por esto decidimo invertir los valores de las columnas de superficie total y superficie cubierta en aquellos casos que la primera es inferior a la segunda

#### Trabajo de hipotesis sobre: conseguir m2 a partir de valor de la propiedad y valor por metro

In [ ]:
# Solo puedo averiguar mi incognita si tengo metros y valor por metro
# ejemplo: x = df['price_aprox_usd']/df['price_usd_per_m2']

#Buscamos las diferentes combinaciones
print("USD, Con Precio y PPM USD pero sin M2: {}".format(
    df.loc[(~df["price_aprox_usd"].isnull()) & (~df["price_usd_per_m2"].isnull()) & (df["surface_total_in_m2"].isnull()),"operation"].count()
))
print("ARS, Con Precio y PPM ARS pero sin M2: {}".format(
    df.loc[(~df["price_aprox_local_currency"].isnull()) & (~df["price_per_m2"].isnull()) & (df["surface_total_in_m2"].isnull()),"operation"].count()
))
print("Con Precio default y PPM default pero sin M2: {}".format(
    df.loc[(~df["price"].isnull()) & (~df["price_per_m2"].isnull()) & (df["surface_total_in_m2"].isnull()),"operation"].count()
))
print("Con Precio default y PPM USD pero sin M2: {}".format(
    df.loc[(~df["price"].isnull()) & (~df["price_usd_per_m2"].isnull()) & (df["surface_total_in_m2"].isnull()),"operation"].count()
))

# Hipotesis fallada, no sirve para tener nuevos datos

#### Trabajo de hipotesis sobre: encontrar en titulo y descripcion valores utiles para rellenar nulos en M2

In [9]:
#creamos una regex  y la corremos en titulo y en descripcion para ver que encuentra
pattern= r'([\.\d]{2,99}) (?!m²|m2|mt|metro)'
m2ExtractedFromTitle=df.loc[df["surface_total_in_m2"].isnull(),'title'].str.extract(pattern, re.IGNORECASE)
m2FromDescription=df.loc[df["surface_total_in_m2"].isnull(),'description'].str.extract(pattern, re.IGNORECASE)

# imprimir resumen resultados
print("Valores en columna titulo: {}".format(m2ExtractedFromTitle.dropna().describe().loc["count",0]))
print("Valores en columna descripcion: {}".format(m2FromDescription.dropna().describe().loc["count",0]))


# Imprimir lo encontrado en titulo.
df["m2Extracted"] = m2ExtractedFromTitle
for index,x in df.iloc[m2ExtractedFromTitle.dropna().index].loc[:,["title","m2Extracted"]].iterrows():
    print("\r Found: {}  \t Title: {}  ".format(x["m2Extracted"],x["title"]))

    
# Dropeamos columna temporal
df.drop("m2Extracted",axis=1,inplace=True)

#Hipotesis fallada, tras revisar los resultados, hay muchas informacion falsa y no es confiable

Valores en columna titulo: 936
Valores en columna descripcion: 7318


IndexError: positional indexers are out-of-bounds

###  2. Normalizar, corregir y rellenar información

** Los metros cuadrados que encontramos invertidos los corregimos

In [6]:
#Creo columna temporal_dos para filtrar subconjunto de datos relevantes a invertir
df['temporal_dos'] = (df.surface_total_in_m2 < df.surface_covered_in_m2)
print("Cantidad de registros a invertir entre Superficies total y cubierta: {}".format(df['temporal_dos'].sum()))

#Creo columna temporal para guardar datos
df['temporal'] = df.surface_total_in_m2 

#Paso valores de superficie cubierta a superficie total
df.loc[df['temporal_dos'],'surface_total_in_m2'] = df.loc[df['temporal_dos'],'surface_covered_in_m2']

#Paso valores de superficie total a superficie cubierta
df.loc[df['temporal_dos'], 'surface_covered_in_m2'] = df.loc[df['temporal_dos'], 'temporal']

#Recreamos la columna temporal para ver si siguen existiendo valores invertidos
df['temporal_dos'] = (df.surface_total_in_m2 < df.surface_covered_in_m2)
print("Cantidad de registros que siguen invertidos: {}".format(df['temporal_dos'].sum()))

#Dropeamos las temporales
df.drop('temporal', axis=1)
df.drop('temporal_dos', axis=1);

Cantidad de registros a invertir entre Superficies total y cubierta: 1106
Cantidad de registros que siguen invertidos: 0


** limpieza de nulls en variables de superficie

In [7]:
# Los metros en cero en superficie totales los ponemos en null
print("Valores M2 cubierto en cero puestos en Nan: {}".format((df["surface_covered_in_m2"] == 0).sum()))
print("Valores M2 totales en cero puestos en Nan: {}".format((df["surface_total_in_m2"] == 0).sum()))
df.loc[(df["surface_total_in_m2"] == 0),["surface_total_in_m2"]] = np.nan
df.loc[(df["surface_covered_in_m2"] == 0),["surface_covered_in_m2"]] = np.nan
print("-------------")

#Revisamos valores antes del eemplazo
print("Antes del reemplazo")
print("Nulos en totales: {}".format(df['surface_total_in_m2'].isnull().sum()))
print("Nulos en cubiertos: {}".format(df['surface_covered_in_m2'].isnull().sum()))
print("Nulos en ambos al mismo tiempo: {}".format(df.loc[(df['surface_covered_in_m2'].isnull()) & (df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Nulos totales y no en cubiertos: {}".format(df.loc[(~df['surface_covered_in_m2'].isnull()) & (df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Nulos cubierto y no en totales: {}".format(df.loc[(df['surface_covered_in_m2'].isnull()) & (~df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Valores iguales: {}".format(df.loc[df['surface_total_in_m2'] == df['surface_covered_in_m2'],"surface_covered_in_m2"].count()))


# relleno los m2totales faltantes con los cubiertos
df.loc[(~df['surface_covered_in_m2'].isnull()) & ( df['surface_total_in_m2'].isnull()) ,"surface_total_in_m2"] = df["surface_covered_in_m2"]
# relleno los m2cubiertos faltantes con los totales
df.loc[( df['surface_covered_in_m2'].isnull()) & (~df['surface_total_in_m2'].isnull()) ,"surface_covered_in_m2"] = df["surface_total_in_m2"]
print("-------------")

#Revisamos valores despues del eemplazo
print("Despues del reemplazo")
print("Nulos en totales: {}".format(df['surface_total_in_m2'].isnull().sum()))
print("Nulos en cubiertos: {}".format(df['surface_covered_in_m2'].isnull().sum()))
print("Nulos en ambos al mismo tiempo: {}".format(df.loc[(df['surface_covered_in_m2'].isnull()) & (df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Nulos totales y no en cubiertos: {}".format(df.loc[(~df['surface_covered_in_m2'].isnull()) & (df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Nulos cubierto y no en totales: {}".format(df.loc[(df['surface_covered_in_m2'].isnull()) & (~df['surface_total_in_m2'].isnull()) ,:].loc[:,"operation"].count()))
print("Valores iguales: {}".format(df.loc[df['surface_total_in_m2'] == df['surface_covered_in_m2'],"surface_covered_in_m2"].count()))


Valores M2 cubierto en cero puestos en Nan: 2
Valores M2 totales en cero puestos en Nan: 383
-------------
Antes del reemplazo
Nulos en totales: 39711
Nulos en cubiertos: 19909
Nulos en ambos al mismo tiempo: 12752
Nulos totales y no en cubiertos: 26959
Nulos cubierto y no en totales: 7157
Valores iguales: 24173
-------------
Despues del reemplazo
Nulos en totales: 12752
Nulos en cubiertos: 12752
Nulos en ambos al mismo tiempo: 12752
Nulos totales y no en cubiertos: 0
Nulos cubierto y no en totales: 0
Valores iguales: 58289


### 3. Quitar todo lo que no nos sirve

### Quitamos duplicados

In [8]:
# Muestro forma inicial.
display(df.shape)

# buscar indices de registros duplicados (sin tener en cuenta las urls y la 1er columna de autonumerico)
duplicados=df.loc[df.drop("Unnamed: 0",axis=1).drop("properati_url",axis=1).drop("image_thumbnail",axis=1).duplicated(keep="last")]
print("Registros duplicados: {}".format(duplicados["operation"].count()))


# DROP duplicados
df.drop(duplicados.index, inplace=True)
display(df.shape)

(121220, 28)

Registros duplicados: 5082


(116138, 28)

### Quitamos columnas redundantes, que no agregan al modelo de prediccion

In [ ]:
# BORRADO DE COLUMNAS SIN USO
def drop_column(column,df):
    try:
        df.drop(column,axis=1,inplace=True)
        print("Dropeando columna {} ".format(column));
    except:
        print("Columna {} ya dropeada ".format(column)) ;
    
# Muestro forma inicial.
display(df.shape)

# properati_url: no tiene ningun uso de valor predictivo
drop_column("properati_url",df)

# image_thumbnail: como se vió antes, hay imagenes duplicadas para departamentos distintos.
drop_column("image_thumbnail",df)

# unnamed 0: replica el indice en cada linea
drop_column("Unnamed: 0",df)

# operation: siempre es venta, no suma nada al modelo
drop_column("operation",df)

# country_name: siempres es argentina, no suma a modelo
drop_column("country_name",df)


display(df.shape)